<a href="https://colab.research.google.com/github/NazarioR9/BNBR_Challenge/blob/master/notebooks/TranslateWithTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs

In [ ]:
!pip install transformers --quiet

# Imports

In [ ]:
import os, sys, gc
import random
import pandas as pd
import numpy as np

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
from transformers import MarianTokenizer, MarianMTModel
from typing import List

In [ ]:
from tqdm.auto import tqdm

# Envs

In [ ]:
seed = 2020

In [ ]:
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available(): 
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# Read Data

In [ ]:
path = './../data/'

In [ ]:
train = pd.read_csv(path+'final_train.csv')
test = pd.read_csv(path+'final_test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

,ID,text
0,02V56KMO,how to overcome bad feelings and emotions
1,03BMGTOK,i feel like giving up in life
2,03LZVFM6,i was so depressed feel like got no strength ...
3,0EPULUM5,i feel so low especially since i had no one t...
4,0GM4C5GD,can i be successful when i am a drug addict ?


# Utilities

In [ ]:
class TranslateDataset(Dataset):
  def __init__(self, df):
    super(TranslateDataset, self).__init__()
    self.df = df

  def  __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    return self.df.loc[idx, 'text']

In [ ]:
class TranslateLoader:
  def __init__(self, ds, bs):
    self.ds = ds
    self.bs = bs
    self.count = 0
    self.szs = len(ds)
    self.maxs = self.szs//bs
    if self.szs % bs != 0:
      self.maxs += 1

  def __len__(self):
    return self.maxs

  def __iter__(self):
    return self

  def __next__(self):
    if self.count >= self.maxs:
      raise StopIteration
    
    idxs = list(range(self.count*self.bs, min( (self.count+1)*self.bs, self.szs ) ))
    batch = [ self.ds[i] for i in idxs]
    self.count += 1

    return batch

#Translation

In [ ]:
src = 'en'  # source language
trg = 'fr'  # target language
mname = f'Helsinki-NLP/opus-mt-{src}-{trg}'

In [ ]:
model = MarianMTModel.from_pretrained(mname)
tok = MarianTokenizer.from_pretrained(mname)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_marian.py:97: UserWarning: Recommended: pip install mosestokenizer
  warnings.warn("Recommended: pip install mosestokenizer")


In [ ]:
ds_train = TranslateDataset(train)
ds_test = TranslateDataset(test)

dl_train = TranslateLoader(ds_train, bs=32)
dl_test = TranslateLoader(ds_test, bs=32)

In [ ]:
def translate(dl):
  translated = []

  for _ in tqdm(range(len(dl)), desc='Translation'):
    data = next(dl)
    batch = tok.prepare_translation_batch(src_texts=data)
    gen = model.generate(**batch)
    translated += tok.batch_decode(gen, skip_special_tokens=True)

  return translated

In [ ]:
translated_train = translate(dl_train)

In [ ]:
translated_test = translate(dl_test)

In [ ]:
len(translated_train), len(train)

(597, 597)

In [ ]:
train[f'translation_{src}_to_{trg}'] = translated_train
test[f'translation_{src}_to_{trg}'] = translated_test

In [ ]:
train.head()

,ID,text,label,Depression,Alcohol,Suicide,Drugs,translation_en_to_fr
0,SUAVK39Z,i feel that it was better i die am happy,0,1,0,0,0,J'ai l'impression que c'était mieux que je meu...
1,9JDAGUV3,why do i get hallucinations ?,3,0,0,0,1,Pourquoi ai-je des hallucinations?
2,419WR1LQ,i am stressed due to lack of financial suppor...,0,1,0,0,0,Je suis stressé en raison du manque de soutien...
3,6UY7DX6Q,why is life important ?,2,0,0,1,0,Pourquoi la vie est-elle importante?
4,FYC0FTFB,how could i be helped to go through the depre...,0,1,0,0,0,Comment pourrais-je être aidé à traverser la d...


In [ ]:
test.head()

,ID,text,translation_en_to_fr
0,02V56KMO,how to overcome bad feelings and emotions,comment surmonter les mauvais sentiments et le...
1,03BMGTOK,i feel like giving up in life,J'ai envie d'abandonner dans la vie.
2,03LZVFM6,i was so depressed feel like got no strength ...,J'étais tellement déprimé que j'avais l'impres...
3,0EPULUM5,i feel so low especially since i had no one t...,Je me sens si faible d'autant plus que je n'av...
4,0GM4C5GD,can i be successful when i am a drug addict ?,Puis-je réussir quand je suis toxicomane?


# Save everyrhing

In [ ]:
train.to_csv(path + 'train_translated_to_french.csv', index=False)

In [ ]:
test.to_csv(path + 'test_translated_to_french.csv', index=False)